顺序模型是多个网络层的线性堆叠。
可以通过将网络层实例的列表传递给 Sequential 的构造器，来创建一个 Sequential 模型

In [1]:
from keras.models import Sequential
from keras.layers import Dense,Activation

Using TensorFlow backend.
G:\anaconda\envs\svd\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
G:\anaconda\envs\svd\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
G:\anaconda\envs\svd\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
G:\anaconda\envs\svd\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarn

In [2]:
model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])

使用 .add() 方法将各层添加到模型中
model = Sequential() 
model.add(Dense(32, input_dim=784)) 
model.add(Activation('relu')) 

### 指定数据数据的尺寸
模型需要知道它所期望的输入的尺寸。出于这个原因，顺序模型中的第一层（且只有第一层，因为下面的层可以自动地推断尺寸）需要接收关于其输入尺寸的信息有几种方法来做到这一点：

1. 传递一个 input_shape 参数给第一层。它是一个表示尺寸的元组 (一个由整数或 None 组成的元组，其中 None 表示可能为任何正整数)。在 input_shape 中不包含数据的 batch 大小。
2. 某些 2D 层，例如 Dense，支持通过参数 input_dim 指定输入尺寸，某些 3D 时序层支持 input_dim 和 input_length 参数。
3. 如果需要为输入指定一个固定的 batch 大小（这对 stateful RNNs 很有用），可以传递一个 batch_size 参数给一个层。如果同时将 batch_size=32 和 input_shape=(6, 8) 传递给一个层，那么每一批输入的尺寸就为 (32，6，8)。

In [3]:
# 相互等价
model = Sequential()
model.add(Dense(32, input_shape=(784,)))

model = Sequential()
model.add(Dense(32, input_dim=784))

### 模型编译

在训练模型之前，需要配置学习过程，这是通过 compile 方法完成的。它接收三个参数:
1. 优化器 optimizer。它可以是现有优化器的字符串标识符，如 rmsprop 或 adagrad，也可以是 Optimizer 类的实例。详见：optimizers。
2. 损失函数 loss，模型试图最小化的目标函数。它可以是现有损失函数的字符串标识符，如 categorical_crossentropy 或 mse，也可以是一个目标函数。详见：losses。
3. 评估标准 metrics。对于任何分类问题，你都希望将其设置为 metrics = ['accuracy']。评估标准可以是现有的标准的字符串标识符，也可以是自定义的评估标准函数。详见: metrics。

In [4]:
# 多分类问题
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 二分类问题
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 均方误差回归问题
model.compile(optimizer='rmsprop',
              loss='mse')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
# 自定义评估标准函数
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', mean_pred])

### 模型训练

Keras 模型在输入数据和标签的 Numpy 矩阵上进行训练。为了训练一个模型，通常会使用 fit 函数

In [22]:
# 对于具有 2 个类的单输入模型（二进制分类）
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 生成虚拟数据
import numpy as np
# 生成100行50列的随机浮点数，浮点数范围：（0，1）
data = np.random.random((100000,100))
print(data)
# 1000行一列的随机整数，从[0,2)中随机。
labels = np.random.randint(2,size=(100000,1))
# print(labels)

[[0.81114717 0.54430669 0.92214077 ... 0.58463751 0.04591163 0.80275721]
 [0.78632847 0.82752616 0.40176934 ... 0.87243341 0.23562903 0.12965371]
 [0.22739238 0.65281481 0.75487748 ... 0.57653252 0.03560155 0.45524497]
 ...
 [0.55479198 0.11386484 0.83163382 ... 0.82650023 0.89804265 0.77701583]
 [0.88937889 0.98513487 0.71108787 ... 0.65311702 0.7611169  0.58549989]
 [0.60986389 0.67876253 0.09262888 ... 0.76559535 0.90793814 0.77895379]]


In [24]:
# 训练模型，以 32 个样本为一个 batch 进行迭代
model.fit(data, labels, epochs=10, batch_size=64)

Epoch 1/10
100000/100000 [==============================] - 1s 12us/step - loss: 0.7146 - acc: 0.5016
Epoch 2/10
100000/100000 [==============================] - 1s 12us/step - loss: 0.7174 - acc: 0.5024
Epoch 3/10
100000/100000 [==============================] - 1s 12us/step - loss: 0.7196 - acc: 0.5032
Epoch 4/10
100000/100000 [==============================] - 1s 12us/step - loss: 0.7225 - acc: 0.5048
Epoch 5/10
100000/100000 [==============================] - 1s 12us/step - loss: 0.7245 - acc: 0.5051
Epoch 6/10
100000/100000 [==============================] - 1s 12us/step - loss: 0.7274 - acc: 0.5030
Epoch 7/10
100000/100000 [==============================] - 1s 12us/step - loss: 0.7325 - acc: 0.5017
Epoch 8/10
100000/100000 [==============================] - 1s 12us/step - loss: 0.7353 - acc: 0.5023
Epoch 9/10
100000/100000 [==============================] - 1s 12us/step - loss: 0.7376 - acc: 0.5016
Epoch 10/10
100000/100000 [==============================] - 1s 12us/step - loss: 

In [27]:
# 对于具有 10 个类的单输入模型（多分类分类）：
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# 生成虚拟数据
import numpy as np
# 生成100行50列的随机浮点数，浮点数范围：（0，1）
data = np.random.random((100000,100))
print(data)
# 1000行一列的随机整数，从[0,2)中随机。
labels = np.random.randint(2,size=(100000,1))
# print(labels)

[[0.27500166 0.37741841 0.51323292 ... 0.84086741 0.7319264  0.34362979]
 [0.01480752 0.61368608 0.9591014  ... 0.07518512 0.91525397 0.89685458]
 [0.7973043  0.70810389 0.02301502 ... 0.1609338  0.85142117 0.85356621]
 ...
 [0.41559723 0.66301536 0.3606279  ... 0.79984085 0.10532354 0.99618585]
 [0.63641397 0.91118035 0.18729435 ... 0.79570787 0.55322488 0.98201734]
 [0.96901101 0.83375116 0.60700095 ... 0.99142752 0.48127263 0.0889728 ]]


In [29]:
import keras
#  将标签转换为分类的 one-hot 编码
one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

# 训练模型，以 32 个样本为一个 batch 进行迭代
model.fit(data, one_hot_labels, epochs=10, batch_size=32)

Epoch 1/10
100000/100000 [==============================] - 3s 32us/step - loss: 0.7025 - acc: 0.4999
Epoch 2/10
100000/100000 [==============================] - 2s 23us/step - loss: 0.6943 - acc: 0.5046
Epoch 3/10
100000/100000 [==============================] - 2s 23us/step - loss: 0.6936 - acc: 0.5037
Epoch 4/10
100000/100000 [==============================] - 2s 22us/step - loss: 0.6933 - acc: 0.5055
Epoch 5/10
100000/100000 [==============================] - 2s 23us/step - loss: 0.6933 - acc: 0.5041
Epoch 6/10
100000/100000 [==============================] - 2s 24us/step - loss: 0.6931 - acc: 0.5062
Epoch 7/10
100000/100000 [==============================] - 4s 36us/step - loss: 0.6929 - acc: 0.5081
Epoch 8/10
100000/100000 [==============================] - 3s 28us/step - loss: 0.6928 - acc: 0.5110
Epoch 9/10
100000/100000 [==============================] - 3s 26us/step - loss: 0.6926 - acc: 0.5123
Epoch 10/10
100000/100000 [==============================] - 3s 29us/step - loss: 

### 基于多层感知器 (MLP) 的 softmax 多分类：

In [30]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

In [41]:
#  生成虚拟数据
import numpy as np
# 1000行 20列
x_train = np.random.random((10000, 20))
# [0，10) 整数
y_train = keras.utils.to_categorical(np.random.randint(10, size=(10000, 1)), num_classes=10)
x_test = np.random.random((1000, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)

model = Sequential()

1. Dense(64) 是一个具有 64 个隐藏神经元的全连接层。
2. 在第一层必须指定所期望的输入数据尺寸：
3. 在这里，是一个 20 维的向量。

In [45]:
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [46]:
#  设定优化器参数
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [47]:
# 配置编译器参数
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [48]:
# 训练
model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)

Epoch 1/20
10000/10000 [==============================] - 1s 78us/step - loss: 2.3042 - acc: 0.0976
Epoch 2/20
10000/10000 [==============================] - 0s 24us/step - loss: 2.3032 - acc: 0.0969
Epoch 3/20
10000/10000 [==============================] - 0s 23us/step - loss: 2.3027 - acc: 0.1089
Epoch 4/20
10000/10000 [==============================] - 0s 24us/step - loss: 2.3026 - acc: 0.1024
Epoch 5/20
10000/10000 [==============================] - 0s 23us/step - loss: 2.3034 - acc: 0.1009
Epoch 6/20
10000/10000 [==============================] - 0s 23us/step - loss: 2.3030 - acc: 0.1017
Epoch 7/20
10000/10000 [==============================] - 0s 23us/step - loss: 2.3031 - acc: 0.0991
Epoch 8/20
10000/10000 [==============================] - 0s 24us/step - loss: 2.3032 - acc: 0.1005
Epoch 9/20
10000/10000 [==============================] - 0s 24us/step - loss: 2.3026 - acc: 0.0960
Epoch 10/20
10000/10000 [==============================] - 0s 24us/step - loss: 2.3025 - acc: 0.1028

In [49]:
#  评估
score = model.evaluate(x_test, y_test, batch_size=128)

1000/1000 [==============================] - 0s 173us/step
